# Poisson model
Create Poisson model for goals scored for each team, see https://www.pinnacle.com/en/betting-articles/soccer/how-to-calculate-poisson-distribution


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline 

data_path = "./data/" 
dic = {}

cols = ["Div","Date","HomeTeam","AwayTeam",
       "FTHG","FTAG","FTR","HTHG","HTAG","HTR"
]

matches = pd.DataFrame(None,columns=cols) # Empty dataframe we'll add the data to

english_leagues = ["prem","champ","league"]

for f in [x for x in os.listdir(data_path)]: #if "prem" in x]:
    if f.split("_")[0] in english_leagues:
        print(f)
        results_df = pd.read_csv(data_path+f,usecols=cols,parse_dates=True)
        # kill empty last rows, below tests this isn't deleting any matches
        results_df = results_df.dropna(how="all")

        # premiership, 20 teams play each other twice, 20*19 total matches
        if f.split("_")[0] == "prem": 
            assert len(results_df) == 20*19
        # lower leagues, 24 teams play 2x, 24*23 total
        else:
            assert len(results_df) == 24*23 

        results_df["Season"] = (f.split("_")[-1].split(".")[0])
        results_df["Season"] = results_df["Season"]

        matches = pd.concat([results_df,matches])

# reorder data by date
matches["Date"] = pd.to_datetime(matches["Date"],dayfirst=True)
matches = matches.sort_values(by="Date")
matches = matches.reset_index(drop=True)
print("==="*3)
print(matches.head())

prem_0910.csv
prem_1314.csv
league_1_0910.csv
champ_0910.csv
league_2_1213.csv
league_2_1112.csv
champ_1314.csv
prem_1617.csv
league_1_1112.csv
league_1_1213.csv
league_2_1011.csv
league_2_1516.csv
league_2_1415.csv
league_1_1011.csv
league_1_1415.csv
league_2_0910.csv
prem_1011.csv
prem_1213.csv
league_2_1617.csv
champ_1011.csv
champ_1213.csv
champ_1516.csv
league_1_1617.csv
prem_1112.csv
champ_1415.csv
champ_1112.csv
champ_1617.csv
league_1_1314.csv
league_1_1516.csv
league_2_1314.csv
prem_1415.csv
prem_1516.csv
           AwayTeam       Date Div FTAG FTHG FTR HTAG HTHG HTR       HomeTeam  \
0  Sheffield United 2009-08-07  E1    0    0   D    0    0   D  Middlesbrough   
1         Peterboro 2009-08-08  E1    1    2   H    0    1   H          Derby   
2           Swansea 2009-08-08  E1    1    2   H    1    0   A      Leicester   
3      Bristol City 2009-08-08  E1    2    2   D    0    0   D        Preston   
4         Blackpool 2009-08-08  E1    1    1   D    1    0   A            Q

# Train model

In [2]:
### PARAMETERS
train_start_date = pd.to_datetime("01/08/2016",dayfirst=True)
predict_start_date = pd.to_datetime("01/03/2017",dayfirst=True)

# Get model training matches after train_start_date, before predict_start_date
starting_season = matches.loc[matches["Date"] > train_start_date].iloc[0]["Season"]
train_season_matches = \
    matches.loc[(matches["Date"] > train_start_date) & (matches["Date"] < predict_start_date)]
# # Get matches to test model 
predict_season_matches = matches.loc[matches["Date"] > predict_start_date]
predict_season_matches = predict_season_matches.loc[predict_season_matches["Div"] == "E0"]

## Initialise attack/def for teams
team_stats = {}
for team in matches["HomeTeam"].unique():
    if team not in list(team_stats.keys()):
        # Prevents crashing when 2 newly promoted teams play
        # each other, as probability matrix has two most likely scores
        # Should probably fix this at some point
        noise = np.random.normal(0, 0.01,1 )
        team_stats[team] = {"home_att" : 1+noise, 
                            "home_def" : 1+noise,
                            "away_att" : 1+noise,
                            "away_def" : 1+noise}

### TRAINING
# 1 - Find total goals scored home/away for each league in training set
league_goals = {}
for div in train_season_matches["Div"].unique():
    div_matches = train_season_matches.loc[train_season_matches["Div"] == div]
    league_goals[div] = {}
    league_goals[div]["Home goals scored"] = div_matches["FTHG"].sum()/len(div_matches)
    league_goals[div]["Away goals scored"] = div_matches["FTAG"].sum()/len(div_matches)
    # Conceded is simply the above swapped
    league_goals[div]["Home goals conceded"] = league_goals[div]["Away goals scored"]
    league_goals[div]["Away goals conceded"] = league_goals[div]["Home goals scored"]

    
# Run through all teams and their goals in training season, update attack/defence ratings
for team in pd.concat([train_season_matches,predict_season_matches])["HomeTeam"].unique():
    
    home_matches = train_season_matches.loc[train_season_matches["HomeTeam"] == team]
    away_matches = train_season_matches.loc[train_season_matches["AwayTeam"] == team]
    
    # Team is present in training set, if not then they must be promoted so ignore
    # this leaves att/def as default vals of 1.0, 1.0    
    if len(set(home_matches["Div"])) == 1:
        starting_division = home_matches["Div"].iloc[0]
        total_home_goals = home_matches["FTHG"].sum()/len(home_matches)
        team_stats[team]["home_att"] = \
            total_home_goals/league_goals[starting_division]["Home goals scored"]
        
        total_away_goals = away_matches["FTAG"].sum()/len(away_matches)
        team_stats[team]["away_att"] = \
            total_away_goals/league_goals[starting_division]["Away goals scored"]

        total_home_conceded = home_matches["FTAG"].sum()/len(home_matches)
        team_stats[team]["home_def"] = \
            total_home_conceded/league_goals[starting_division]["Home goals conceded"]
        
        total_away_conceded = away_matches["FTHG"].sum()/len(away_matches)
        team_stats[team]["away_def"] = \
            total_away_conceded/league_goals[starting_division]["Away goals conceded"]
        
        #print(team,team_stats[team])

# Testing

A Skellam model is used to predict home/draw/away result. A traditional Poisson model is shown at the end of the cell and commented out, this can be used for precise scoreline prediction.

In [8]:
import math
from scipy.stats import skellam

def poisson(mu,k):
    return np.exp(-mu) * (mu**k) / math.factorial(k)

# number of season simulations to run 
iters = 500 
# list of premiership teams, used for creating dataframes
prem_teams = predict_season_matches["HomeTeam"].unique() 
# holds results, we will append to this
simulation_results = pd.DataFrame(index=prem_teams) 

for j in range(iters):
    
    if j % 50 == 0:    print("{}% done".format(100*j/iters))
    
    table = {team : 0 for team in prem_teams} # initialise league table 
    
    for ix, match in predict_season_matches.iterrows():
        home_team = match["HomeTeam"]
        away_team = match["AwayTeam"]
        div = match["Div"]

        home_stats = team_stats[home_team]
        away_stats = team_stats[away_team]
        league_stats = league_goals[div]

        # Mean home goals = Home Att * Away Def * Avg goals for division
        expected_home_goals = \
            home_stats["home_att"]*away_stats["away_def"]*league_stats["Home goals scored"]
        expected_away_goals = \
            home_stats["home_def"]*away_stats["away_att"]*league_stats["Away goals scored"]    

        ## skellam distribution method - poisson dist. convolutions
        rng = range(-5,6,1)
        
        skel = skellam.pmf(rng,expected_home_goals,expected_away_goals)
        away_prob = (skel[0:5].sum())
        draw_prob = (skel[6].sum())
        home_prob = (skel[7:].sum())

        probabilities = \
            np.array([home_prob, draw_prob, away_prob])/(home_prob+draw_prob+away_prob)
        result = np.random.choice(["H","D","A"],p=probabilities)


        #Update league table
        if result == "H":
            table[home_team] += 3
        elif result == "A":
            table[away_team] += 3
        elif result == "D":
            table[home_team] += 1
            table[away_team] += 1

    # add this simulated season results to our records
    table = pd.DataFrame(table,index=range(1)).T.sort_values(by=0,ascending=False)
    table.columns = [j]
    simulation_results = pd.concat([simulation_results,table],axis=1)

    
    # poisson distribution method
#     max_goals = 5
#     home_goals_dist = np.array([poisson(expected_home_goals,k) for k in range(max_goals+1)])
#     away_goals_dist = np.array([poisson(expected_away_goals,k) for k in range(max_goals+1)])

#     prob_matrix = np.outer(home_goals_dist,away_goals_dist) # HOME GOALS = ROWS
    
# #     print(list(np.hstack(np.where(prob_matrix == prob_matrix.max()))))
#     (home_goals, away_goals) = tuple(np.hstack(np.where(prob_matrix == prob_matrix.max())))
    
#     print(home_goals,away_goals)
    
    # check against pinnacle.com poisson example
#     if home_team == "Tottenham" and away_team == "Everton": #
#         print(pd.DataFrame(prob_matrix))
#         print(home_goals,away_goals)
        #         print(expected_home_goals,expected_away_goals) # yep fine
#         for k in range(5):
#             print(poisson(expected_home_goals,k)) # also fine
#             print(poisson(expected_away_goals,k)) # FINE

0.0% done
10.0% done
20.0% done
30.0% done
40.0% done
50.0% done
60.0% done
70.0% done
80.0% done
90.0% done


In [4]:
# Take mean over all simulations
simulated_table = \
    simulation_results.mean(axis=1).astype(int).sort_values(ascending=False).reset_index()
simulated_table.columns = ["Sim_team","Sim_pts"]

In [5]:
# Creates actual results table from the real 2016-2017 premiership season, prints comparison
s
actual_table = {team: 0 for team in prem_teams}

for index,match in predict_season_matches[["HomeTeam","AwayTeam","FTR"]].iterrows():

    home_team = match["HomeTeam"]
    away_team = match["AwayTeam"]
    result = match["FTR"]

    if result == "H":
        actual_table[home_team] += 3
    elif result == "A":
        actual_table[away_team] += 3
    elif result == "D":
        actual_table[home_team] += 1
        actual_table[away_team] += 1
        
actual_table = \
    pd.DataFrame(actual_table,index=range(1)).T.sort_values(by=0,ascending=False).reset_index()
actual_table.columns = ["Act_team","Act_pts"]
res = pd.concat([simulated_table,actual_table],axis=1)
print(res)

          Sim_team  Sim_pts        Act_team  Act_pts
0        Tottenham       27       Tottenham       33
1          Chelsea       26         Chelsea       30
2         Man City       25       Liverpool       27
3        Liverpool       24        Man City       26
4          Arsenal       24         Arsenal       25
5       Man United       23      Man United       21
6          Everton       21       Leicester       20
7        West Brom       15     Bournemouth       20
8         West Ham       14  Crystal Palace       19
9            Stoke       13         Everton       17
10     Southampton       13         Swansea       17
11         Swansea       12     Southampton       16
12       Leicester       12            Hull       13
13         Burnley       12           Stoke       12
14     Bournemouth       12        West Ham       12
15   Middlesbrough       11         Burnley        9
16  Crystal Palace       11         Watford        9
17      Sunderland       10   Middlesbrough   